In [22]:
import pandas as pd
import lightgbm as lgb
from L_eddie_preprocessing import *
from L_eddie_modeling import *

In [23]:
data = pd.read_csv('data/output/20250908_modeling_data.csv')
data = downcast(data)

data.info()

70.5% 압축됨
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1621812 entries, 0 to 1621811
Data columns (total 42 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   월ID                         1621812 non-null  int8   
 1   상점ID                        1621812 non-null  int8   
 2   상품ID                        1621812 non-null  int16  
 3   월별 판매량                      1621812 non-null  int8   
 4   상품분류ID                      1621812 non-null  int8   
 5   채널                          1621812 non-null  int8   
 6   도시                          1621812 non-null  int8   
 7   쇼핑몰 종류                      1621812 non-null  int8   
 8   상품대분류                       1621812 non-null  int8   
 9   상품소분류                       1621812 non-null  int8   
 10  상품_경과개월                     1621812 non-null  int8   
 11  상품_신상여부                     1621812 non-null  int8   
 12  상품_상점_경과개월                  1621812 non-null  

In [24]:
data.head()

,월ID,상점ID,상품ID,월별 판매량,상품분류ID,채널,도시,쇼핑몰 종류,상품대분류,상품소분류,...,상품ID_월별_평균_판매량_lag3,상품ID_도시_월별_평균_판매량_lag1,상품ID_도시_월별_평균_판매량_lag2,상품ID_도시_월별_평균_판매량_lag3,상품ID_상품분류ID_월별_평균_판매량_lag1,상품ID_상품분류ID_월별_평균_판매량_lag2,상품ID_상품분류ID_월별_평균_판매량_lag3,상품ID_쇼핑몰_종류_월별_평균_판매량_lag1,상품ID_쇼핑몰_종류_월별_평균_판매량_lag2,상품ID_쇼핑몰_종류_월별_평균_판매량_lag3
0,17,2,27,1,19,1,2,5,5,12,...,1.000000,1.0,0.0,0.0,1.0000,1.000000,1.000000,1.000000,1.000000,1.000000
1,5,2,30,1,40,1,2,5,11,6,...,8.674418,1.0,0.0,0.0,2.5000,3.823529,8.674418,2.583333,4.050000,8.880000
2,15,2,30,1,40,1,2,5,11,6,...,2.148148,1.0,1.0,0.0,1.9375,1.333333,2.148148,2.000000,1.230769,2.142857
3,16,2,30,1,40,1,2,5,11,6,...,1.333333,1.0,1.0,1.0,1.5000,1.937500,1.333333,1.250000,2.000000,1.230769
4,34,2,30,0,40,1,2,5,11,6,...,1.000000,1.0,1.0,1.0,1.0000,1.000000,1.000000,1.000000,1.000000,1.000000


In [25]:
data.columns

Index(['월ID', '상점ID', '상품ID', '월별 판매량', '상품분류ID', '채널', '도시', '쇼핑몰 종류',
       '상품대분류', '상품소분류', '상품_경과개월', '상품_신상여부', '상품_상점_경과개월', '상품_상점_신상여부', '연',
       '월', '분기', '월날짜', '월별_판매량_lag1', '월별_판매량_lag2', '월별_판매량_lag3',
       '월별_판매량_mean3', '월별_판매량_diff_lag1_2', '월별_판매량_diff_lag2_3',
       '월별_판매건수_lag1', '월별_판매건수_lag2', '월별_판매건수_lag3', '월별_평균_판매가_lag1',
       '월별_평균_판매가_lag2', '월별_평균_판매가_lag3', '상품ID_월별_평균_판매량_lag1',
       '상품ID_월별_평균_판매량_lag2', '상품ID_월별_평균_판매량_lag3', '상품ID_도시_월별_평균_판매량_lag1',
       '상품ID_도시_월별_평균_판매량_lag2', '상품ID_도시_월별_평균_판매량_lag3',
       '상품ID_상품분류ID_월별_평균_판매량_lag1', '상품ID_상품분류ID_월별_평균_판매량_lag2',
       '상품ID_상품분류ID_월별_평균_판매량_lag3', '상품ID_쇼핑몰_종류_월별_평균_판매량_lag1',
       '상품ID_쇼핑몰_종류_월별_평균_판매량_lag2', '상품ID_쇼핑몰_종류_월별_평균_판매량_lag3'],
      dtype='object')

In [26]:
categorical_features = ['상점ID', '상품ID', '상품분류ID', '채널', '도시', '쇼핑몰 종류',
       '상품대분류', '상품소분류', '상품_신상여부', '상품_상점_신상여부', '연','월', '분기']

numeric_features = ['상품_경과개월', '상품_상점_경과개월','월별_판매량_lag1', '월별_판매량_lag2', '월별_판매량_lag3',
       '월별_판매량_mean3', '월별_판매량_diff_lag1_2', '월별_판매량_diff_lag2_3',
       '월별_판매건수_lag1', '월별_판매건수_lag2', '월별_판매건수_lag3', '월별_평균_판매가_lag1',
       '월별_평균_판매가_lag2', '월별_평균_판매가_lag3', '상품ID_월별_평균_판매량_lag1',
       '상품ID_월별_평균_판매량_lag2', '상품ID_월별_평균_판매량_lag3', '상품ID_도시_월별_평균_판매량_lag1',
       '상품ID_도시_월별_평균_판매량_lag2', '상품ID_도시_월별_평균_판매량_lag3',
       '상품ID_상품분류ID_월별_평균_판매량_lag1', '상품ID_상품분류ID_월별_평균_판매량_lag2',
       '상품ID_상품분류ID_월별_평균_판매량_lag3', '상품ID_쇼핑몰_종류_월별_평균_판매량_lag1',
       '상품ID_쇼핑몰_종류_월별_평균_판매량_lag2', '상품ID_쇼핑몰_종류_월별_평균_판매량_lag3']

date_column = '월날짜'
target_column = '월별 판매량'

In [27]:
model_data = data[categorical_features + numeric_features + [date_column, target_column]]

In [28]:
valid_days = 30
test_days = 30

In [29]:
determine_data_periods(model_data, date_column, validation_days= valid_days, test_days= test_days)

{'train_range': ('2013-04-01', '2015-08-31'),
 'valid_range': ('2015-09-01', '2015-10-01'),
 'test_range': ('2015-10-02', '2015-11-01')}

In [30]:
X_train, y_train, X_val, y_val, X_test, y_test= data_split(model_data, date_column, target_column, train_range=('2013-04-01', '2015-09-02') , valid_range = ('2015-09-01', '2015-10-02'), test_range=('2015-10-03', '2015-11-02'), drop_col=None)

print(f"Train: {X_train.shape}, {y_train.shape}")
print(f"Validation: {X_val.shape}, {y_val.shape}")
print(f"Test: {X_test.shape}, {y_test.shape}")

c:\Users\dodor.HAEWON\OneDrive\바탕 화면\Basic_machine_learning\L_eddie_preprocessing.py:349: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[ts_col] = pd.to_datetime(df[ts_col])


Train: (1376601, 39), (1376601,)
Validation: (60324, 39), (60324,)
Test: (214200, 39), (214200,)


In [31]:
import lightgbm as lgb
from lightgbm import LGBMRegressor
import pandas as pd
from sklearn.metrics import mean_squared_error


# 2) Your params (with correct key: force_col_wise)
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 255,
    'learning_rate': 0.005,
    'feature_fraction': 0.75,
    'bagging_fraction': 0.75,
    'force_col_wise': True,  # corrected name
    'random_state': 10,
    'n_estimators': 1500,    # mirrors previous num_boost_round
}

model = LGBMRegressor(**params)

# 3) Callbacks for early stopping + logging
callbacks = [
    lgb.early_stopping(stopping_rounds=150, first_metric_only=True, verbose=True),
    lgb.log_evaluation(period=100),
]

# 4) Fit
model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    eval_metric='rmse',
    callbacks=callbacks,
    categorical_feature=categorical_features,  # optional if dtype is category
)

# 5) Predict using best_iteration
y_pred = model.predict(X_test, num_iteration=model.best_iteration_)


[LightGBM] [Warning] feature_fraction is set=0.75, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.75
[LightGBM] [Warning] bagging_fraction is set=0.75, subsample=1.0 will be ignored. Current value: bagging_fraction=0.75
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] feature_fraction is set=0.75, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.75
[LightGBM] [Warning] bagging_fraction is set=0.75, subsample=1.0 will be ignored. Current value: bagging_fraction=0.75
[LightGBM] [Info] Total Bins 15247
[LightGBM] [Info] Number of data points in the train set: 1376601, number of used features: 39
[LightGBM] [Warning] feature_fraction is set=0.75, colsample_bytree=1

In [42]:
import numpy as np
import pandas as pd

# 예측값 클립
y_pred_clip = np.clip(np.asarray(y_pred).ravel(), 0, 20)
test_data = pd.read_csv('data/output/rawData/test.csv')

# 예측 DF (키 + 예측)
pred_df = pd.DataFrame({
    '상점ID': X_test['상점ID'].astype(int).values,
    '상품ID': X_test['상품ID'].astype(int).values,
    'item_cnt_month': y_pred_clip
})

# test_data 키 준비(순서 유지)
keys = test_data[['ID','상점ID','상품ID']].copy()
keys['상점ID'] = keys['상점ID'].astype(int)
keys['상품ID'] = keys['상품ID'].astype(int)

# 왼쪽 기준 병합 + 순서 유지
submission = keys.merge(pred_df, on=['상점ID','상품ID'], how='left', sort=False)[['ID','item_cnt_month']]
submission['item_cnt_month'] = submission['item_cnt_month'].fillna(0)

submission.to_csv('data/output/submission.csv', index=False)


In [43]:
keys
submission

,ID,item_cnt_month
0,0,1.687530
1,1,1.155853
2,2,2.367427
3,3,1.344799
4,4,1.667259
...,...,...
214195,214195,1.121307
214196,214196,1.017823
214197,214197,1.031007
214198,214198,0.978944
